In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [0]:
from pyspark.sql.types import *
import  pyspark.sql.functions as F

In [0]:
# File location and type
file_location = "/FileStore/tables/patients.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
patients = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", True) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
# File location and type
file_location = "/FileStore/tables/conditions.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
conditions = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", True) \
  .option("sep", delimiter) \
  .load(file_location)

##### 1. Qual é quantidade de pessoas do género feminino e masculino e a sua percentagem sobre o total de doentes?

##### 1.1. Crie uma visualização com esta informação (gráfico de barras)

In [0]:
genero_doentes = patients.groupBy("GENDER").agg(F.count("*").alias("Total"))
total_doentes = patients.select("Id").distinct().count()

genero_com_percentagem = genero_doentes.withColumn(
    "Percentagem M/F",(genero_doentes["Total"] / total_doentes * 100))

genero_com_percentagem.display()

GENDER,Total,Percentagem M/F
F,6253,50.62338082901554
M,6099,49.376619170984455


Databricks visualization. Run in Databricks to view.

##### 2. Identifique se existe informação de doentes com data de nascimento superior à data de morte.

In [0]:
patients = patients.withColumn("BIRTHDATE", F.to_date(F.col("BIRTHDATE")))
patients = patients.withColumn("DEATHDATE", F.to_date(F.col("DEATHDATE")))

data_nascimento_superior = patients.filter(F.col("BIRTHDATE") > F.col("DEATHDATE"))

##### 3. Calcule a idade( em anos) das pessoas usando as seguintes condições:

##### 3.1. Se o estiver morto, essa será a data final para calcular a idade

##### 3.2. Se estiver vivo, considere como data final, 2020-04-05 para o cálculo da idade

In [0]:
idade_doente = (
    F.when(F.col("DEATHDATE").isNotNull(),
           F.floor(F.datediff(F.col("DEATHDATE"), F.col("BIRTHDATE")) / 365.25))
    .otherwise(F.floor(F.datediff(F.to_date(F.lit("2020-04-05")), F.col("BIRTHDATE")) / 365.25))
    .alias("IDADE")
)


patientes_idade = patients.select('id','BIRTHDATE','DEATHDATE', idade_doente)

##### 4. Identifique a idade máxima, idade mínima, a média, mediana1 e máximo.

In [0]:
idades_doentes_estat = (
    patientes_idade.agg(
        F.max(F.col("IDADE")).alias("Idade_Maxima"),
        F.min(F.col("IDADE")).alias("Idade_Minima"),
        F.avg(F.col("IDADE")).alias("Idade_Media"),
        F.expr("percentile(IDADE, 0.5)").alias("Idade_Mediana")
    )
)

idades_doentes_estat.show()

+------------+------------+-----------------+-------------+
|Idade_Maxima|Idade_Minima|      Idade_Media|Idade_Mediana|
+------------+------------+-----------------+-------------+
|         110|          -1|44.16337435233161|         45.0|
+------------+------------+-----------------+-------------+



In [0]:
patients_sem_idade_negativa = patientes_idade.filter(F.col("IDADE") != -1)

In [0]:
idades_doentes_estat = (
    patients_sem_idade_negativa.agg(
        F.max(F.col("IDADE")).alias("Idade_Maxima"),
        F.min(F.col("IDADE")).alias("Idade_Minima"),
        F.avg(F.col("IDADE")).alias("Idade_Media"),
        F.expr("percentile(IDADE, 0.5)").alias("Idade_Mediana")
    )
)

idades_doentes_estat.show()

+------------+------------+-----------------+-------------+
|Idade_Maxima|Idade_Minima|      Idade_Media|Idade_Mediana|
+------------+------------+-----------------+-------------+
|         110|           0|44.23295224195248|         45.0|
+------------+------------+-----------------+-------------+



##### 5. Faça um histograma com 100 bins (intervalos) da idade das pessoas.

In [0]:
patients_sem_idade_negativa.display()

id,BIRTHDATE,DEATHDATE,IDADE
f0f3bc8d-ef38-49ce-a2bd-dfdda982b271,2017-08-24,null,2
067318a4-db8f-447f-8b6e-f2f61e9baaa5,2016-08-01,null,3
ae9efba3-ddc4-43f9-a781-f72019388548,1992-06-30,null,27
199c586f-af16-4091-9998-ee4cfc02ee7a,2004-01-09,null,16
353016ea-a0ff-4154-85bb-1cf8b6cedf20,1996-11-15,null,23
f58bf921-cba1-475a-b4f8-dc6fa3b8f89c,2019-06-12,null,0
b9fd2dd8-181b-494b-ab15-e9f286d668d9,1983-12-12,null,36
d22592ac-552f-4ecd-a63d-7663d77ce9ba,1989-06-22,null,30
28658715-b770-4576-9a81-fbb2282a98ea,1991-07-31,null,28
055ae6fc-7e18-4a39-8058-64082ca6d515,2005-01-16,null,15


Databricks visualization. Run in Databricks to view.

##### 6. Como estão distribuídas cada umas das etnias sobre o total dos doentes?

In [0]:
patients.select("ETHNICITY").show()

+-----------+
|  ETHNICITY|
+-----------+
|nonhispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|   hispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|   hispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|nonhispanic|
|   hispanic|
+-----------+
only showing top 20 rows



##### 7. Qual é raça com maior e menor número de doentes e qual é o % total sobre o total da população?

In [0]:
etnia_doentes = (
    patients.groupBy("ETHNICITY").agg(
    F.count("*").alias("Total_Por_Etnia"),
    (F.count("*") / total_doentes * 100).alias("Porcentagem_Por_Etnia")
    ))

etnia_doentes.show()

+-----------+---------------+---------------------+
|  ETHNICITY|Total_Por_Etnia|Porcentagem_Por_Etnia|
+-----------+---------------+---------------------+
|nonhispanic|          11036|    89.34585492227978|
|   hispanic|           1316|   10.654145077720207|
+-----------+---------------+---------------------+



##### 8. Quais são 15 condições mais detetadas?

##### 8.1. Faça um horizontal barplot com esta informação?

In [0]:
condicoes_doentes = (
    conditions.groupBy("DESCRIPTION")
    .agg(F.count("*").alias("Nº_casos"))
    .orderBy(F.desc("Nº_casos"))
    .limit(15)
)

condicoes_doentes.display()

DESCRIPTION,Nº_casos
Suspected COVID-19,9106
COVID-19,8820
Fever (finding),8083
Cough (finding),6202
Body mass index 30+ - obesity (finding),5002
Loss of taste (finding),4711
Prediabetes,3917
Anemia (disorder),3650
Fatigue (finding),3516
Hypertension,3168


Databricks visualization. Run in Databricks to view.

##### 9. Identifique quantos códigos nas condições estão repetidos?

#####9.1. Quantas descrições diferentes tem cada um dos códigos identificados?

#####9.2. Proponha uma forma de unificar os códigos e a suas descrições.

In [0]:
codigos_repetidos = (
    conditions.groupBy("CODE")
    .agg(F.count("*").alias("N_Codigos_repetidos"))
    .filter(F.col("N_Codigos_repetidos") > 1)
    .orderBy(F.desc("N_Codigos_repetidos"))
)

codigos_repetidos.show(truncate=False)

+---------+-------------------+
|CODE     |N_Codigos_repetidos|
+---------+-------------------+
|840544004|9106               |
|840539006|8820               |
|386661006|8083               |
|49727002 |6202               |
|162864005|5002               |
|36955009 |4711               |
|15777000 |3917               |
|271737000|3650               |
|84229001 |3516               |
|59621000 |3168               |
|248595008|2970               |
|40055000 |2655               |
|19169002 |2212               |
|233604007|2000               |
|389087006|1867               |
|271825005|1867               |
|56018004 |1816               |
|267036007|1816               |
|444814009|1577               |
|55822004 |1540               |
+---------+-------------------+
only showing top 20 rows



In [0]:
descricoes_codigo = (
    conditions.groupBy("CODE")
    .agg(F.countDistinct("DESCRIPTION").alias("Nº_Descricoes"))
    .orderBy(F.desc("Nº_Descricoes"))
)

descricoes_codigo.show(truncate=False)

+--------------+-------------+
|CODE          |Nº_Descricoes|
+--------------+-------------+
|233604007     |2            |
|427089005     |2            |
|74400008      |1            |
|449868002     |1            |
|38822007      |1            |
|254637007     |1            |
|47693006      |1            |
|60951000119105|1            |
|195967001     |1            |
|161622006     |1            |
|56018004      |1            |
|36955009      |1            |
|75498004      |1            |
|47505003      |1            |
|363406005     |1            |
|840544004     |1            |
|48333001      |1            |
|16114001      |1            |
|275272006     |1            |
|65275009      |1            |
+--------------+-------------+
only showing top 20 rows



In [0]:
unir_codigos = (
    conditions.groupBy("CODE")
    .agg(F.first("DESCRIPTION").alias("Descricao_Unificada"))
)

conditions_code = conditions.join(unir_codigos, on="CODE", how="left")

conditions_code.show(truncate=False)

+---------+----------+----------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|CODE     |START     |STOP      |PATIENT                             |ENCOUNTER                           |DESCRIPTION                         |Descricao_Unificada                 |
+---------+----------+----------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|65363002 |2019-02-15|2019-08-01|f0f3bc8d-ef38-49ce-a2bd-dfdda982b271|d5ee30a9-362f-429e-a87a-ee38d999b0a5|Otitis media                        |Otitis media                        |
|65363002 |2019-10-30|2020-01-30|f0f3bc8d-ef38-49ce-a2bd-dfdda982b271|8bca6d8a-ab80-4cbf-8abb-46654235f227|Otitis media                        |Otitis media                        |
|386661006|2020-03-01|2020-03-30|f0f3bc8d-ef38-49ce-a2bd-dfdda982b271|681c380b-3c84-4c55-8

##### 10. Calcule a duração das condições(doenças) que os doentes padecem. desde a primeira vez que foi diagnosticado.

##### 10.1. Considere que para as pessoas mortas, a data de finalização da condição é o dia da morte específico para cada um dos doentes.

##### 10.2. Calcule a média em dias e anos, se for mais de 365 dias transforme a anos.

In [0]:
result_df = conditions.join(patients, (patients.Id == conditions.PATIENT), "inner")

result_df = result_df.select("ID","FIRST","LAST", "DEATHDATE", "DESCRIPTION", "START", "STOP")

In [0]:
result_df = result_df.withColumn("START", F.col("START").cast("date"))
result_df = result_df.withColumn("STOP", F.col("STOP").cast("date"))
result_df = result_df.withColumn("DEATHDATE", F.col("DEATHDATE").cast("date"))

result_df = result_df.withColumn("CONDITION_STOP", F.when(F.col("DEATHDATE").isNotNull(), F.col("DEATHDATE")).otherwise(F.col("STOP")))

result_df = result_df.withColumn("DURACAO_DIAS", F.expr("datediff(CONDITION_STOP, START)"))

In [0]:
media_dias = result_df.select(F.avg("DURACAO_DIAS")).collect()[0][0]

media_anos = media_dias / 365

print(f"Média em dias: {media_dias:.0f} dias")
print(f"Média em anos: {media_anos:.0f} anos")

Média em dias: 1672 dias
Média em anos: 5 anos


##### 11. O Dr Anthony Fauci recebeu informação afirmando que o número de doenças crónicas está relacionado diretamente com estádios mais severos do covid-19. A indicação dele é que toda condição detetada que tiver mais de 1 ano será considerada como uma doença crónica.

##### 12. Quantas doenças/condições foram classificadas como crónicas segundo a conceito do Dr Fauci.

In [0]:
result_df = result_df.withColumn("CONDICAO_CRONICA",
                                 F.expr("CASE WHEN DURACAO_DIAS > 365 \
                                        THEN 'Sim' \
                                        ELSE 'Não' END"))

num_condicoes_cronicas = result_df.filter(F.col("CONDICAO_CRONICA") == "Sim").count()

print(f"Segundo a conceito do Dr Fauci, foram classificadas como crónicas {num_condicoes_cronicas} doenças")

Segundo a conceito do Dr Fauci, foram classificadas como crónicas 14079 doenças


##### 13. Identifique a duração mínima, máxima e média ( em anos) das doenças que crónicas.

In [0]:
duracao_minima_anos = result_df.select(F.expr("MIN(DURACAO_DIAS) / 365")).collect()[0][0]
duracao_maxima_anos = result_df.select(F.expr("MAX(DURACAO_DIAS) / 365")).collect()[0][0]
duracao_media_anos = result_df.select(F.expr("AVG(DURACAO_DIAS) / 365")).collect()[0][0]

print(f"Duração mínima das doenças crônicas: {duracao_minima_anos:.0f} anos")
print(f"Duração máxima das doenças crônicas: {duracao_maxima_anos:.0f} anos")
print(f"Duração média das doenças crônicas: {duracao_media_anos:.0f} anos")

Duração mínima das doenças crônicas: 0 anos
Duração máxima das doenças crônicas: 108 anos
Duração média das doenças crônicas: 5 anos


##### 14. Qual é o nome das 10 pessoas com mais doenças crónicas.

In [0]:
contagem_doencas_cronicas = result_df.filter(F.col("CONDICAO_CRONICA") == "Sim") \
    .groupBy("ID", "FIRST", "LAST").count() \
    .withColumnRenamed("count", "NUM_DOENCAS_CRONICAS")

top10_doencas_cronicas = contagem_doencas_cronicas \
    .orderBy(F.col("NUM_DOENCAS_CRONICAS").desc()).limit(10)

top10_doencas_cronicas.show()

+--------------------+------------+-----------+--------------------+
|                  ID|       FIRST|       LAST|NUM_DOENCAS_CRONICAS|
+--------------------+------------+-----------+--------------------+
|eb69309e-b9d0-448...|     Robin66|  Mayert710|                  25|
|7b41338d-5b8f-4ca...|Jefferson174| Lubowitz58|                  23|
|3e5af47c-897c-4ed...|  Antione404| Farrell962|                  22|
|4be710e6-6352-448...|  Shemeka786|Franecki195|                  21|
|176991b1-9555-439...|Nathaniel596|   Terry864|                  21|
|1308e3c0-023b-4b0...|     Clay913| Gerlach374|                  20|
|1b982bad-94c9-4cd...|    Zulma701|    Veum823|                  20|
|bff0121d-4b20-487...|     Raul814|   Robel940|                  20|
|972c7a80-848d-491...|    Ollie731|    Conn188|                  19|
|ca4d4d38-c10e-411...|      Gil594|   Thiel172|                  19|
+--------------------+------------+-----------+--------------------+



In [0]:
# File location and type
file_location = "/FileStore/tables/observations.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
observations = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", True) \
  .option("sep", delimiter) \
  .load(file_location)

##### 15. Identifique qual é o código que indica o peso do doente.

In [0]:
codigo_peso = observations.filter((observations['DESCRIPTION'] == 'Body Weight'))

codigo_peso.select('CODE', 'DESCRIPTION').show()

+-------+-----------+
|   CODE|DESCRIPTION|
+-------+-----------+
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
|29463-7|Body Weight|
+-------+-----------+
only showing top 20 rows



In [0]:
codigo_altura = observations.filter((observations['DESCRIPTION'] == 'Body Height'))

codigo_altura.select('CODE', 'DESCRIPTION').show()

+------+-----------+
|  CODE|DESCRIPTION|
+------+-----------+
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
|8302-2|Body Height|
+------+-----------+
only showing top 20 rows



##### 16. Calcule o BMI (IMC) número

In [0]:
df_bmi = observations.filter((F.col("DESCRIPTION") == "Body Weight") | (F.col("DESCRIPTION") == "Body Height"))

In [0]:
df_new = df_bmi.groupBy("DATE", "PATIENT", "ENCOUNTER").pivot("DESCRIPTION").agg(F.expr("first(VALUE)"))

df_imc = df_new.withColumn("BMI", F.round(F.col("Body Weight") / (F.col("Body Height") / 100) ** 2, 1))

df_imc.select("PATIENT","Body Weight","Body Height", "BMI").show()

+--------------------+-----------+-----------+----+
|             PATIENT|Body Weight|Body Height| BMI|
+--------------------+-----------+-----------+----+
|ce84e142-5627-485...|       12.5|       90.7|15.2|
|247b8bd6-3c9c-4bc...|       15.2|       96.6|16.3|
|f15d55d1-215d-4ae...|       13.4|       87.2|17.6|
|ca695308-d24c-45c...|       14.2|       83.6|20.3|
|efcd46dd-90de-4f6...|        8.3|       68.7|17.6|
|b4787882-c2ae-41b...|       28.3|      141.0|14.2|
|93322fca-edc1-4b2...|       63.3|      166.0|23.0|
|e8fea247-6550-49c...|       75.1|      184.5|22.1|
|7975c012-60c8-491...|       81.3|      185.6|23.6|
|15942763-709e-41a...|       60.2|      172.2|20.3|
|7405088b-ae85-420...|       98.6|      181.0|30.1|
|44083318-4459-473...|        5.8|       59.0|16.7|
|5398dc77-d675-40b...|       13.8|       88.8|17.5|
|9fbc67d4-aca4-4c2...|        5.6|       56.6|17.5|
|9fbc67d4-aca4-4c2...|        8.7|       66.1|19.9|
|18b89e6f-f024-40b...|       87.5|      178.3|27.5|
|08fc5e6c-5a

In [0]:
df_imc_classified = df_imc.withColumn(
    "BMI_Classification",
    F.expr("""
        CASE
            WHEN BMI < 18.5 THEN 'Underweight'
            WHEN BMI >= 18.5 AND BMI <= 24.9 THEN 'Healthy Weight'
            WHEN BMI >= 25.0 AND BMI <= 29.9 THEN 'Overweight'
            WHEN BMI >= 30.0 AND BMI <= 39.9 THEN 'Obesity'
            WHEN BMI >= 40.0 THEN 'Class 3 Obesity'
            ELSE 'Unknown'
        END
    """)
)

df_imc_classified.select('PATIENT','BMI_Classification').show()

+--------------------+------------------+
|             PATIENT|BMI_Classification|
+--------------------+------------------+
|ce84e142-5627-485...|       Underweight|
|247b8bd6-3c9c-4bc...|       Underweight|
|f15d55d1-215d-4ae...|       Underweight|
|ca695308-d24c-45c...|    Healthy Weight|
|efcd46dd-90de-4f6...|       Underweight|
|b4787882-c2ae-41b...|       Underweight|
|93322fca-edc1-4b2...|    Healthy Weight|
|e8fea247-6550-49c...|    Healthy Weight|
|7975c012-60c8-491...|    Healthy Weight|
|15942763-709e-41a...|    Healthy Weight|
|7405088b-ae85-420...|           Obesity|
|44083318-4459-473...|       Underweight|
|5398dc77-d675-40b...|       Underweight|
|9fbc67d4-aca4-4c2...|       Underweight|
|9fbc67d4-aca4-4c2...|    Healthy Weight|
|18b89e6f-f024-40b...|        Overweight|
|08fc5e6c-5a35-4b6...|    Healthy Weight|
|a1550552-dd9d-4bb...|    Healthy Weight|
|a1550552-dd9d-4bb...|    Healthy Weight|
|6632933d-6cc7-4b4...|           Obesity|
+--------------------+------------

In [0]:
stddev_bmi = df_imc_classified.selectExpr("stddev(BMI) as STD_DEV").first()["STD_DEV"]

df_anomalias = df_imc_classified.withColumn(
    "Limite_Superior",
    F.round(F.log(F.col("BMI")) + (3 * stddev_bmi), 2)
).withColumn(
    "Limite_Inferior",
    F.round(F.log(F.col("BMI")) - (3 * stddev_bmi), 2)
)

df_anomalias = df_anomalias.withColumn(
    "Anomalia_Peso",
    F.expr("""
        CASE
            WHEN BMI > Limite_Superior OR BMI < Limite_Inferior THEN true
            ELSE false
        END
    """)
)

df_anomalias.select('PATIENT', "Limite_Superior", "Limite_Inferior", "Anomalia_Peso").show()

+--------------------+---------------+---------------+-------------+
|             PATIENT|Limite_Superior|Limite_Inferior|Anomalia_Peso|
+--------------------+---------------+---------------+-------------+
|ce84e142-5627-485...|          18.27|         -12.83|        false|
|247b8bd6-3c9c-4bc...|          18.34|         -12.76|        false|
|f15d55d1-215d-4ae...|          18.42|         -12.68|        false|
|ca695308-d24c-45c...|          18.56|         -12.54|         true|
|efcd46dd-90de-4f6...|          18.42|         -12.68|        false|
|b4787882-c2ae-41b...|          18.21|          -12.9|        false|
|93322fca-edc1-4b2...|          18.69|         -12.42|         true|
|e8fea247-6550-49c...|          18.65|         -12.46|         true|
|7975c012-60c8-491...|          18.71|         -12.39|         true|
|15942763-709e-41a...|          18.56|         -12.54|         true|
|7405088b-ae85-420...|          18.96|         -12.15|         true|
|44083318-4459-473...|          18